In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random

# Load your dataset from the specified file path
data = pd.read_csv('C:/Users/vishn/OneDrive/Documents/Machine Learning/Vishnu_phd.csv')

# Select features and targets (assuming your CSV file has these columns)
X = data[['SlopeHeight', 'SlopeAngle', 'UCS', 'GSI', 'mi', 'D', 'PoissonsRatio', 'E', 'Density']]
y = data[['FoS', 'SeismicFoS']]

# Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the ANN model
def create_ann(layer_1_neurons, layer_2_neurons, layer_3_neurons, learning_rate):
    # Ensure the number of neurons is valid (at least 1 neuron)
    layer_1_neurons = max(1, int(layer_1_neurons))
    layer_2_neurons = max(1, int(layer_2_neurons))
    layer_3_neurons = max(1, int(layer_3_neurons))

    # Ensure the learning rate is valid (a small positive float)
    learning_rate = max(1e-5, min(learning_rate, 1e-2))

    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))  # Use Input layer to specify input shape
    model.add(Dense(layer_1_neurons, activation='relu'))
    model.add(Dense(layer_2_neurons, activation='relu'))
    model.add(Dense(layer_3_neurons, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for FoS and SeismicFoS
    
    # Use the learning rate in the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

# Check if FitnessMax and Individual classes already exist to avoid overwriting them
if not hasattr(creator, "FitnessMax"):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))

if not hasattr(creator, "Individual"):
    creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the function to evaluate model accuracy
def evaluate(individual):
    # Extract hyperparameters from the individual
    layer_1_neurons = individual[0]
    layer_2_neurons = individual[1]
    layer_3_neurons = individual[2]
    learning_rate = individual[3]
    
    # Build and compile the model with these hyperparameters
    model = create_ann(layer_1_neurons, layer_2_neurons, layer_3_neurons, learning_rate)
    
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    
    # Evaluate the model on the validation set
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    
    return (mae,)  # Return the mean absolute error as the fitness value

# Create the genetic algorithm components
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 32, 128)  # Neurons in hidden layers (32 to 128)
toolbox.register("attr_float", random.uniform, 0.0001, 0.01)  # Learning rate (small positive float)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int, toolbox.attr_int, toolbox.attr_int, toolbox.attr_float), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Create the population
population = toolbox.population(n=10)

# Apply the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=10, verbose=True)

# Extract the best individual after optimization
best_individual = tools.selBest(population, 1)[0]
print(f"Best Individual: {best_individual}")

# Build and train the optimized model
best_layer_1 = best_individual[0]
best_layer_2 = best_individual[1]
best_layer_3 = best_individual[2]
best_learning_rate = best_individual[3]

optimized_model = create_ann(best_layer_1, best_layer_2, best_layer_3, best_learning_rate)

# Train the model with the optimized parameters
optimized_model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=1)

# Evaluate the model
loss, mae = optimized_model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {mae}")


gen	nevals
0  	10    
1  	8     
2  	7     
3  	6     
4  	10    
5  	7     
6  	7     
7  	3     
8  	8     
9  	5     
10 	5     
Best Individual: [36, 88, 0.0009372925022918303, 0.000401121213836317]
Epoch 1/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - loss: 4.8396 - mae: 1.8595
Epoch 2/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 3.8353 - mae: 1.6771
Epoch 3/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - loss: 3.5031 - mae: 1.5619
Epoch 4/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - loss: 3.3901 - mae: 1.4996
Epoch 5/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 3.1038 - mae: 1.4314
Epoch 6/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - loss: 3.2377 - mae: 1.4295
Epoch 7/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - loss: 3.3116 - mae: 1.4309
Epoch 8/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - loss: 3.2260 - mae: 1.4278
Epoch 9/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - loss: 2.8660 - mae: 1.3344
Epoch 10/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predict on both training and test sets
y_pred_train = optimized_model.predict(X_train)
y_pred_test = optimized_model.predict(X_test)

# Split y_train and y_test
y_train_fos = y_train.iloc[:, 0]
y_train_seismicfos = y_train.iloc[:, 1]
y_test_fos = y_test.iloc[:, 0]
y_test_seismicfos = y_test.iloc[:, 1]

# Split predictions
y_pred_train_fos = y_pred_train[:, 0]
y_pred_train_seismicfos = y_pred_train[:, 1]
y_pred_test_fos = y_pred_test[:, 0]
y_pred_test_seismicfos = y_pred_test[:, 1]

# === FoS Metrics ===
r2_fos_train = r2_score(y_train_fos, y_pred_train_fos)
rmse_fos_train = np.sqrt(mean_squared_error(y_train_fos, y_pred_train_fos))
mae_fos_train = mean_absolute_error(y_train_fos, y_pred_train_fos)
mse_fos_train = mean_squared_error(y_train_fos, y_pred_train_fos)

r2_fos_test = r2_score(y_test_fos, y_pred_test_fos)
rmse_fos_test = np.sqrt(mean_squared_error(y_test_fos, y_pred_test_fos))
mae_fos_test = mean_absolute_error(y_test_fos, y_pred_test_fos)
mse_fos_test = mean_squared_error(y_test_fos, y_pred_test_fos)

# === SeismicFoS Metrics ===
r2_seismic_train = r2_score(y_train_seismicfos, y_pred_train_seismicfos)
rmse_seismic_train = np.sqrt(mean_squared_error(y_train_seismicfos, y_pred_train_seismicfos))
mae_seismic_train = mean_absolute_error(y_train_seismicfos, y_pred_train_seismicfos)
mse_seismic_train = mean_squared_error(y_train_seismicfos, y_pred_train_seismicfos)

r2_seismic_test = r2_score(y_test_seismicfos, y_pred_test_seismicfos)
rmse_seismic_test = np.sqrt(mean_squared_error(y_test_seismicfos, y_pred_test_seismicfos))
mae_seismic_test = mean_absolute_error(y_test_seismicfos, y_pred_test_seismicfos)
mse_seismic_test = mean_squared_error(y_test_seismicfos, y_pred_test_seismicfos)

# === Print all metrics ===
print("\n📊 Evaluation Metrics for FoS:")
print(f"Train R²: {r2_fos_train:.6f}, Test R²: {r2_fos_test:.6f}")
print(f"Train RMSE: {rmse_fos_train:.6f}, Test RMSE: {rmse_fos_test:.6f}")
print(f"Train MAE: {mae_fos_train:.6f}, Test MAE: {mae_fos_test:.6f}")
print(f"Train MSE: {mse_fos_train:.6f}, Test MSE: {mse_fos_test:.6f}")

print("\n📊 Evaluation Metrics for SeismicFoS:")
print(f"Train R²: {r2_seismic_train:.6f}, Test R²: {r2_seismic_test:.6f}")
print(f"Train RMSE: {rmse_seismic_train:.6f}, Test RMSE: {rmse_seismic_test:.6f}")
print(f"Train MAE: {mae_seismic_train:.6f}, Test MAE: {mae_seismic_test:.6f}")
print(f"Train MSE: {mse_seismic_train:.6f}, Test MSE: {mse_seismic_test:.6f}")


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

📊 Evaluation Metrics for FoS:
Train R²: 0.979726, Test R²: 0.883237
Train RMSE: 0.164181, Test RMSE: 0.434709
Train MAE: 0.102662, Test MAE: 0.303820
Train MSE: 0.026955, Test MSE: 0.188972

📊 Evaluation Metrics for SeismicFoS:
Train R²: 0.936325, Test R²: 0.755849
Train RMSE: 0.220378, Test RMSE: 0.463939
Train MAE: 0.139395, Test MAE: 0.318322
Train MSE: 0.048567, Test MSE: 0.215239
